In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform


In [2]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')
data.index = data['PatientID']
# drop last 2 columns
data = data.drop(data.columns[-2:], axis=1)

In [3]:
data_target = pd.read_csv('~/Desktop/project_data_new/target_768_avg_expanded.csv')
data_target.index = data_target['Unnamed: 0']
data_target = data_target.drop(['Unnamed: 0'], axis = 1)
# only keep the columns with category in the name
data_target = data_target.loc[:, data_target.columns.str.contains('category')]

In [4]:
data = data[data.index.isin(data_target.index)]

In [5]:
data_target

,category_tnk_1,category_tnk_2,category_tnk_3,category_tnk_4,category_tnk_5,category_tnk_6,category_tnk_7,category_tnk_8,category_tnk_9,category_tnk_10,...,category_stromal_62,category_stromal_63,category_stromal_64,category_stromal_65,category_stromal_66,category_stromal_68,category_stromal_73,category_stromal_78,category_stromal_80,category_stromal_85
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-A6-6650,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-6142,1,1,0,0,0,0,0,0,-1,0,...,1,1,1,1,1,-1,0,0,-1,-1
TCGA-5M-AAT4,-1,-1,-1,0,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-2676,1,1,1,0,1,1,1,1,1,1,...,0,-1,0,-1,0,1,0,1,1,1
TCGA-A6-6652,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-G4-6321,1,1,1,-1,1,1,1,1,-1,1,...,0,0,-1,1,1,-1,1,1,-1,-1
TCGA-RU-A8FL,-1,-1,-1,1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-G4-6303,-1,0,-1,-1,0,0,0,-1,-1,1,...,1,1,1,0,1,-1,0,0,-1,-1


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(1000, 5000),
    'learning_rate': uniform(0.01, 0.1),     # 0.01 to 0.11
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.5, 0.5),          # 0.5 to 1.0
    'colsample_bytree': uniform(0.5, 0.5),   # 0.5 to 1.0
    'gamma': uniform(0, 5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(1, 5),             # Start from 1 to avoid zero regularization
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_b_12 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_b_12 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 15),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5)
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_b_12 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_b_12 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.7323592099410596, 'gamma': 0.03177917514301182, 'learning_rate': 0.10329469651469865, 'max_depth': 10, 'min_child_weight': 5, 'n_estimators': 198, 'subsample': 0.8365191150830908}
Cross-validation F1 scores on training set: [0.50189167 0.41718132 0.49907191 0.39585521 0.52024264]
Average cross-validation F1 score on training set: 0.466848549262388
Classification Report for category_b_12 with 100 features (Train Set):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       120
           1       1.00      1.00      1.00       119
           2       1.00      1.00      1.00       120

    accuracy                           1.00       359
   macro avg       1.00      1.00      1.00       359
weighted avg       1.00      1.00      1.00       359

Classification Report for category_b_12 with 100 features (Test Set)

In [10]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)
# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)

param_dist = {
    'n_estimators': randint(500, 1000),  # Increased upper bound
    'max_depth': randint(2, 8),          # Reduced depth
    'learning_rate': uniform(0.01, 0.09),# Lower learning rates
    'subsample': uniform(0.5, 0.5),      # Range from 0.5 to 1.0
    'colsample_bytree': uniform(0.5, 0.5),
    'min_child_weight': randint(5, 15),  # Increased minimum
    'gamma': uniform(0, 5),              # Expanded range
    'reg_alpha': uniform(0, 1),          # Added L1 regularization
    'reg_lambda': uniform(0, 10),        # Added L2 regularization
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train)
print("Classification Report for category_b_12 with 100 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test)
print("Classification Report for category_b_12 with 100 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.5257393756249946, 'gamma': 1.3932323211830573, 'learning_rate': 0.09174392973699882, 'max_depth': 4, 'min_child_weight': 6, 'n_estimators': 552, 'reg_alpha': 0.489452760277563, 'reg_lambda': 9.856504541106007, 'subsample': 0.6210276357557503}
Cross-validation F1 scores on training set: [0.48863636 0.38843055 0.47293886 0.36705517 0.54819698]
Average cross-validation F1 score on training set: 0.45305158492273556
Classification Report for category_b_12 with 100 features (Train Set):
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       120
           1       0.98      0.97      0.98       119
           2       0.98      0.98      0.98       120

    accuracy                           0.98       359
   macro avg       0.98      0.98      0.98       359
weighted avg       0.98      0.98      0.98       359

Cla

In [11]:
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

# Assuming X and Y_category_b_12 are already defined
X = data.values
Y_category_b_12 = data_target['category_b_12'].values
label_encoder = LabelEncoder()
Y_category_b_12 = label_encoder.fit_transform(Y_category_b_12)

# Stratified split of the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42, stratify=Y_category_b_12)

# Step 1: Initialize XGBoost Classifier and RFE
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
rfe = RFE(estimator=xgb_clf, n_features_to_select=100)  # Choose top 100 features

# Fit RFE to select top 100 features
X_train_rfe = rfe.fit_transform(X_train, Y_train)
X_test_rfe = rfe.transform(X_test)


param_dist = {
    'n_estimators': randint(500, 1000),  # Increased upper bound
    'max_depth': randint(2, 8),          # Reduced depth
    'learning_rate': uniform(0.01, 0.09),# Lower learning rates
    'subsample': uniform(0.5, 0.5),      # Range from 0.5 to 1.0
    'colsample_bytree': uniform(0.5, 0.5),
    'min_child_weight': randint(5, 15),  # Increased minimum
    'gamma': uniform(0, 5),              # Expanded range
    'reg_alpha': uniform(0, 1),          # Added L1 regularization
    'reg_lambda': uniform(0, 10),        # Added L2 regularization
}

# Perform Randomized Search with cross-validation on the selected features
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train_rfe, Y_train)

# Get the best model from Randomized Search
best_xgb = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters from Randomized Search:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_xgb, X_train_rfe, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Step 3: Fit the model on the training data for the final evaluation on test and train sets
best_xgb.fit(X_train_rfe, Y_train)

# Evaluate on the training set
Y_train_pred = best_xgb.predict(X_train_rfe)
print("Classification Report for category_b_12 with 100 selected features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_xgb.predict(X_test_rfe)
print("Classification Report for category_b_12 with 100 selected features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters from Randomized Search: {'colsample_bytree': 0.5866471600354228, 'gamma': 0.7821852133554302, 'learning_rate': 0.03252186083481358, 'max_depth': 7, 'min_child_weight': 10, 'n_estimators': 547, 'reg_alpha': 0.18286599710730733, 'reg_lambda': 9.346139973397097, 'subsample': 0.8191352969216752}
Cross-validation F1 scores on training set: [0.56608155 0.44555012 0.51604938 0.51551307 0.51535495]
Average cross-validation F1 score on training set: 0.5117098133827519
Classification Report for category_b_12 with 100 selected features (Train Set):
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       120
           1       0.98      0.97      0.98       119
           2       0.98      0.98      0.98       120

    accuracy                           0.98       359
   macro avg       0.98      0.98      0.98       359
weighted avg       0.98      0.98      0.98    

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint, uniform

# Assuming 'data' and 'data_target' are your feature matrix and target DataFrame respectively
# X contains feature values
X = data.values

# Set the chunk size (number of target variables per part)
chunk_size = 40

# Loop through target columns in chunks
for chunk_start in range(0, 241, chunk_size):
    # Get the current chunk of target columns
    chunk_end = min(chunk_start + chunk_size, 241)
    target_chunk = data_target.columns[chunk_start:chunk_end]
    
    # Initialize list to store results for the current chunk
    chunk_results = []

    # Open a file to write classification reports for the current chunk
    report_file_path = f"/home/qiuaodon/Desktop/CRC_image/XGboost_results/Classification_Reports_100features_part_{chunk_start}_{chunk_end}.txt"
    with open(report_file_path, "w") as report_file:
        # Iterate over each target column in the chunk
        for target_col in target_chunk:
            # Extract the target column for the current category
            Y = data_target[target_col].values

            # Encode labels if they are not numerical
            label_encoder = LabelEncoder()
            Y_encoded = label_encoder.fit_transform(Y)

            # Initialize the XGBoost Classifier
            xgb_clf = xgb.XGBClassifier(
                objective='multi:softmax', 
                random_state=42, 
                use_label_encoder=False,
                eval_metric='mlogloss'
            )

            # Select top 100 features using RFE
            rfe = RFE(estimator=xgb_clf, n_features_to_select=100)
            X_selected = rfe.fit_transform(X, Y_encoded)
            
            # Split the data into training and testing sets with stratification
            X_train, X_test, Y_train, Y_test = train_test_split(
                X_selected, Y_encoded, test_size=0.2, random_state=42, stratify=Y_encoded
            )

            # Define the hyperparameter distribution for XGBoost
            param_dist = {
                'n_estimators': randint(500, 1000),  # Increased upper bound
                'max_depth': randint(2, 8),          # Reduced depth
                'learning_rate': uniform(0.01, 0.09),# Lower learning rates
                'subsample': uniform(0.5, 0.5),      # Range from 0.5 to 1.0
                'colsample_bytree': uniform(0.5, 0.5),
                'min_child_weight': randint(5, 15),  # Increased minimum
                'gamma': uniform(0, 5),              # Expanded range
                'reg_alpha': uniform(0, 1),          # Added L1 regularization
                'reg_lambda': uniform(0, 10),        # Added L2 regularization
            }

            # Perform Randomized Search with cross-validation
            random_search = RandomizedSearchCV(
                estimator=xgb_clf,
                param_distributions=param_dist,
                n_iter=50, 
                cv=5, 
                scoring='f1_weighted', 
                n_jobs=-1, 
                verbose=1, 
                random_state=42
            )
            random_search.fit(X_train, Y_train)

            # Get the best model from Randomized Search
            best_xgb = random_search.best_estimator_
            best_params = random_search.best_params_

            # Fit the best model on the training data
            best_xgb.fit(X_train, Y_train)

            # Evaluate on the training set
            Y_train_pred = best_xgb.predict(X_train)
            train_report = classification_report(Y_train, Y_train_pred)

            # Evaluate on the test set
            Y_test_pred = best_xgb.predict(X_test)
            test_report_dict = classification_report(Y_test, Y_test_pred, output_dict=True)
            test_report = classification_report(Y_test, Y_test_pred)

            # Write classification reports to the file
            report_file.write(f"Classification Report for {target_col} (Train Set):\n")
            report_file.write(train_report)
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Test Set):\n")
            report_file.write(test_report)
            report_file.write("\n\n" + "="*80 + "\n\n")

            # Calculate and store precision, recall, and accuracy
            precision = precision_score(Y_test, Y_test_pred, average='weighted')
            recall = recall_score(Y_test, Y_test_pred, average='weighted')
            accuracy = accuracy_score(Y_test, Y_test_pred)

            # Extract precision, recall, and f1-score specifically for class '1' (if it exists)
            class_labels = label_encoder.classes_
            class_indices = label_encoder.transform(class_labels)
            class_1_index = np.where(class_indices == 1)[0]

            if class_1_index.size > 0:
                class_1_label = str(class_indices[class_1_index[0]])
                class_1_metrics = test_report_dict.get(class_1_label, {"precision": None, "recall": None, "f1-score": None})
            else:
                class_1_metrics = {"precision": None, "recall": None, "f1-score": None}

            chunk_results.append({
                "Target Variable": target_col,
                "Precision": precision,
                "Recall": recall,
                "Accuracy": accuracy,
                "Class 1 Precision": class_1_metrics["precision"],
                "Class 1 Recall": class_1_metrics["recall"],
                "Class 1 F1-Score": class_1_metrics["f1-score"],
                "Best Hyperparameters": best_params
            })
    
    # Save the results for the current chunk to an Excel file
    results_df = pd.DataFrame(chunk_results)
    results_df.to_excel(
        f"/home/qiuaodon/Desktop/CRC_image/XGboost_results/Precision_Recall_Accuracy_100features_part_{chunk_start}_{chunk_end}.xlsx",
        index=False
    )
